In [ ]:
# !git clone https://github.com/HIT-SCIR/ELMoForManyLangs.git
# !pip install overrides
# !pip install ELMoForManyLangs
# !wget https://3f7xrg.bl.files.1drv.com/y4mzCOdmCDMRHErLHsESWkD0rmmY1j9ca3CfhfCpv6poE3j-0dZd9HmKVC3k0LWif3I2XgyC1tErV8SrVr1mJNVNHYPmU_qqNvvZVBhOijBfsdwWaYVs6Zd4QzsC4HaljGNbTWwtnQ-JrWog9EB0DbblDlKlNBYxcroYpLW9_qrHX7Ub2XEnYVcZ1gqMptzr3Us9Jj66IdrWRLoaYK_FJWiRQ -O dataset.zip
# !unzip -P Ymj26Uv5 dataset.zip
# !pip install transformers

     ---------------------------------------- 0.0/44.1 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.1 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.1 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.1 kB 165.2 kB/s eta 0:00:01
     ----------------------------------- -- 41.0/44.1 kB 219.4 kB/s eta 0:00:01
     -------------------------------------- 44.1/44.1 kB 217.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.0 MB 653.6 kB/s eta 0:00:16
   ---------------------------------------- 0.1/10.0 MB 1.2 MB/s eta 0:00:09
    --------------------------------------- 0.2/10.0 MB 1.8 MB/s eta 0:00:06
   - -------------------------------------- 0.5/10.0 MB 2.7 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/10.0 MB 2.8 MB/s eta 0:00:04
   ----- ---------------------------------- 1.4/10.0 MB 5.0 MB/s eta 0:00:02
   -----

In [1]:
import numpy as np
import pandas as pd
import os
import re
import jieba
import torch
from transformers import BertTokenizer, BertModel

In [2]:
import numpy as np
import jieba
import os
from elmoformanylangs import Embedder

bert_model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)

prefix = os.path.abspath(os.path.join(os.getcwd(), ".."))

answers = {}
text_features = []
text_clf_targets = []
text_reg_targets = []

In [3]:
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = bert_model(**inputs)

    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

In [ ]:
def extract_features(text_features, text_clf_targets, text_reg_targets):
    base_path = os.path.join(os.getcwd(), 'EATD-Corpus')
    topics = ['positive', 'neutral', 'negative']

    indx=0
    for dir_name in os.listdir(base_path):
        indx = indx+1
        
        dir_path = os.path.join(base_path, dir_name)

        if not os.path.isdir(dir_path):
            continue

        answers[indx] = []

        for topic in topics:
            try:
                with open(os.path.join(dir_path, f'{topic}.txt'), 'r', encoding='utf-8') as f:
                    lines = f.read().strip()
                    seg_text_iter = jieba.cut(lines, cut_all=False)
                    segmented_text = ' '.join(seg_text_iter)  # Join segmented words
                    answers[indx].append(segmented_text)
            except FileNotFoundError:
                print(f"File not found: {os.path.join(dir_path, f'{topic}.txt')}")
                continue
        
        try:
            with open(os.path.join(dir_path, 'new_label.txt'), 'r', encoding='utf-8') as fli:
                target = float(fli.readline().strip())
            
            text_clf_targets.append(1 if target >= 53 else 0)
            text_reg_targets.append(target)
            
        except (FileNotFoundError, ValueError) as e:
            print(f"Error reading label: {e}")
            continue

        # Generate BERT embeddings for each segmented answer and calculate mean
        try:
            sentence_embeddings = [get_bert_embeddings(answer) for answer in answers[indx]]
            avg_embeddings = np.mean(sentence_embeddings, axis=0)
            text_features.append(avg_embeddings)
        except Exception as e:
            print(f"Error generating embeddings: {e}")

In [5]:
extract_features(text_features, text_clf_targets, text_reg_targets)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\meena\AppData\Local\Temp\jieba.cache
Loading model cost 1.675 seconds.
Prefix dict has been built successfully.


In [6]:
print("Saving npz file locally...")
np.savez('./Features/TextWhole/mod_samples_reg_avg.npz', text_features)
np.savez('./Features/TextWhole/mod_labels_reg_avg.npz', text_reg_targets)
np.savez('./Features/TextWhole/mod_samples_clf_avg.npz', text_features)
np.savez('./Features/TextWhole/mod_labels_clf_avg.npz', text_clf_targets)

Saving npz file locally...
